In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from tensorflow.keras import layers, models, Input

#Initializing a dataset

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


#MnistClassifierInterface

In [3]:
from abc import ABC, abstractmethod

class MnistClassifierInterface:

    @abstractmethod
    def train(self, x_train, y_train):
        pass

    @abstractmethod
    def predict(self, x_train):
        pass

#RandomForestModel

In [4]:


class RandomForestModel(MnistClassifierInterface):
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100)

    def train(self, x_train: np.ndarray, y_train: np.ndarray):
        x_train_flat = x_train.reshape(x_train.shape[0], -1)
        self.model.fit(x_train_flat, y_train)

    def predict(self, x_test: np.ndarray) -> np.ndarray:
        x_test_flat = x_test.reshape(x_test.shape[0], -1)
        return self.model.predict(x_test_flat)


#Feed-Forward Neural Network Model

In [5]:
class FeedForwardNNModel(MnistClassifierInterface):
    def __init__(self):
        self.model = MLPClassifier(
            hidden_layer_sizes=(128, 64),
            activation='relu',
            solver='adam',
            max_iter=10,
            verbose=True
        )

    def train(self, x_train, y_train):
        x_train_flat = x_train.reshape(x_train.shape[0], -1)
        self.model.fit(x_train_flat, y_train)

    def predict(self, x_test):
        x_test_flat = x_test.reshape(x_test.shape[0], -1)
        predictions = self.model.predict(x_test_flat)
        return predictions

#Convolutional Neural Network Model

In [6]:
class CNNModel(MnistClassifierInterface):
    def __init__(self):
        self.model = models.Sequential([
            Input(shape=(28, 28, 1)),
            layers.Conv2D(32, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])

        self.model.compile(optimizer='adam',
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])

    def train(self, x_train, y_train):
        x_train_exp = x_train[..., np.newaxis]
        self.model.fit(x_train_exp, y_train, epochs=5, batch_size=32, verbose=2)

    def predict(self, x_test):
        x_test_exp = x_test[..., np.newaxis]
        predictions = self.model.predict(x_test_exp)
        return np.argmax(predictions, axis=1)


#MnistClassifier


In [7]:
class MnistClassifier:
    def __init__(self, algorithm: str):
        if algorithm == 'rf':
            self.model = RandomForestModel()
        elif algorithm == 'nn':
            self.model = FeedForwardNNModel()
        elif algorithm == 'cnn':
            self.model = CNNModel()
        else:
            raise ValueError("Invalid algorithm. Use 'rf', 'nn', or 'cnn'.")

    def train(self, x_train, y_train):
        self.model.train(x_train, y_train)

    def predict(self, x_test):
        return self.model.predict(x_test)

#Tests

In [8]:
x_train, x_test = x_train / 255.0, x_test / 255.0

classifier = MnistClassifier(algorithm='rf')
classifier.train(x_train, y_train)
predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')
print(predictions)

Accuracy: 0.9698
[7 2 1 ... 4 5 6]


In [9]:
classifier =  MnistClassifier(algorithm='nn')
classifier.train(x_train, y_train)
predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')
print(predictions)

Iteration 1, loss = 0.39670932
Iteration 2, loss = 0.15847226
Iteration 3, loss = 0.11168397
Iteration 4, loss = 0.08642452
Iteration 5, loss = 0.06970530
Iteration 6, loss = 0.05617617
Iteration 7, loss = 0.04759308
Iteration 8, loss = 0.04026506
Iteration 9, loss = 0.03213286
Iteration 10, loss = 0.02698389


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.9758
[7 2 1 ... 4 5 6]


In [10]:
classifier =  MnistClassifier(algorithm='cnn')
classifier.train(x_train, y_train)
predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')
print(predictions)

Epoch 1/5
1875/1875 - 54s - 29ms/step - accuracy: 0.9572 - loss: 0.1406
Epoch 2/5
1875/1875 - 80s - 43ms/step - accuracy: 0.9855 - loss: 0.0467
Epoch 3/5
1875/1875 - 80s - 43ms/step - accuracy: 0.9898 - loss: 0.0317
Epoch 4/5
1875/1875 - 47s - 25ms/step - accuracy: 0.9925 - loss: 0.0237
Epoch 5/5
1875/1875 - 47s - 25ms/step - accuracy: 0.9944 - loss: 0.0186
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Accuracy: 0.9913
[7 2 1 ... 4 5 6]
